In [1]:
import tensorflow as tf
import os
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn import utils
import plot_utils as pu
import matplotlib.pyplot as plt
import seaborn  as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error as mae
import utils_daniel





In [2]:
X_train_estimated_a = pd.read_parquet('../data/A/X_train_estimated.parquet')
X_train_estimated_b = pd.read_parquet('../data/B/X_train_estimated.parquet')
X_train_estimated_c = pd.read_parquet('../data/C/X_train_estimated.parquet')
X_train_estimated_a['date_forecast'] = pd.to_datetime(X_train_estimated_a['date_forecast'])
X_train_estimated_b['date_forecast'] = pd.to_datetime(X_train_estimated_b['date_forecast'])
X_train_estimated_c['date_forecast'] = pd.to_datetime(X_train_estimated_c['date_forecast'])
X_train_estimated_a = X_train_estimated_a.set_index("date_forecast")
X_train_estimated_b = X_train_estimated_b.set_index("date_forecast")
X_train_estimated_c = X_train_estimated_c.set_index("date_forecast")


X_train_observed_a = pd.read_parquet('../data/A/X_train_observed.parquet')
X_train_observed_b = pd.read_parquet('../data/B/X_train_observed.parquet')
X_train_observed_c = pd.read_parquet('../data/C/X_train_observed.parquet')
X_train_observed_a['date_forecast'] = pd.to_datetime(X_train_observed_a['date_forecast'])
X_train_observed_b['date_forecast'] = pd.to_datetime(X_train_observed_b['date_forecast'])
X_train_observed_c['date_forecast'] = pd.to_datetime(X_train_observed_c['date_forecast'])
X_train_observed_a = X_train_observed_a.set_index("date_forecast")
X_train_observed_b = X_train_observed_b.set_index("date_forecast")
X_train_observed_c = X_train_observed_c.set_index("date_forecast")

X_test_estimated_a = pd.read_parquet('../data/A/X_test_estimated.parquet')
X_test_estimated_b = pd.read_parquet('../data/B/X_test_estimated.parquet')
X_test_estimated_c = pd.read_parquet('../data/C/X_test_estimated.parquet')
X_test_estimated_a['date_forecast'] = pd.to_datetime(X_test_estimated_a['date_forecast'])
X_test_estimated_b['date_forecast'] = pd.to_datetime(X_test_estimated_b['date_forecast'])
X_test_estimated_c['date_forecast'] = pd.to_datetime(X_test_estimated_c['date_forecast'])
X_test_estimated_a = X_test_estimated_a.set_index("date_forecast")
X_test_estimated_b = X_test_estimated_b.set_index("date_forecast")
X_test_estimated_c = X_test_estimated_c.set_index("date_forecast")

A_targets = pd.read_parquet("../data/A/train_targets.parquet")
B_targets = pd.read_parquet("../data/B/train_targets.parquet")
C_targets = pd.read_parquet("../data/C/train_targets.parquet")

A_targets["time"] = pd.to_datetime(A_targets["time"])
B_targets["time"] = pd.to_datetime(B_targets["time"])
C_targets["time"] = pd.to_datetime(C_targets["time"])


A_targets = A_targets.set_index("time")
B_targets = B_targets.set_index("time")
C_targets = C_targets.set_index("time")

# removing rows where pv_measurment remains the same for more than 18 hours.
A_targets_clean = utils_daniel.remove_constant_values(A_targets, "pv_measurement", 18).dropna()
B_targets_clean = utils_daniel.remove_constant_values(B_targets, "pv_measurement", 18).dropna()
C_targets_clean = utils_daniel.remove_constant_values(C_targets, "pv_measurement", 18).dropna()




In [3]:
#periodizing
day = 60*60*24
year = 365.2425*day

A_targets_clean["Seconds"] = A_targets_clean.index.map(pd.Timestamp.timestamp)
#A_targets_clean

A_targets_clean["Day sin"] = np.sin(A_targets_clean["Seconds"] * (2*np.pi / day))
A_targets_clean["Day cos"] = np.cos(A_targets_clean["Seconds"] * (2*np.pi / day))
A_targets_clean["Year sin"] = np.sin(A_targets_clean["Seconds"] * (2*np.pi / year))
A_targets_clean["Year cos"] = np.cos(A_targets_clean["Seconds"] * (2*np.pi / year))
A_targets_clean = A_targets_clean.drop("Seconds", axis=1) #we dont want this in the data, only for calculations
A_targets_clean

,pv_measurement,Day sin,Day cos,Year sin,Year cos
time,,,,,
2019-06-02 22:00:00,0.00,-5.000000e-01,0.866025,0.487287,-0.873242
2019-06-02 23:00:00,0.00,-2.588190e-01,0.965926,0.486661,-0.873591
2019-06-03 00:00:00,0.00,-2.189871e-12,1.000000,0.486034,-0.873940
2019-06-03 01:00:00,0.00,2.588190e-01,0.965926,0.485408,-0.874288
2019-06-03 02:00:00,19.36,5.000000e-01,0.866025,0.484781,-0.874636
...,...,...,...,...,...
2023-04-30 19:00:00,9.02,-9.659258e-01,0.258819,0.881158,-0.472822
2023-04-30 20:00:00,0.00,-8.660254e-01,0.500000,0.880819,-0.473453
2023-04-30 21:00:00,0.00,-7.071068e-01,0.707107,0.880479,-0.474085


In [5]:
#make input data in batches based on window size

X_a, y_a = utils_daniel.df_to_X_y2(A_targets_clean)

print(X_a.shape, y_a.shape)





(31515, 24, 5) (31515,)


In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(X_a, y_a, test_size=0.2, random_state=42)

print(X_train)

def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(64, input_shape=input_shape, activation='relu'))
    model.add(Dense(1))  # Output layer with 1 neuron for regression
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model


# Assuming input_shape is (sequence_length, num_features)
input_shape = (24 , 5)

# Create and train LSTM model for Region A
model_a = create_lstm_model(input_shape)
model_a.fit(X_train, Y_train, epochs=10, batch_size=32, validation_data=(X_test, Y_test))



# Predict solar power for the test set
predictions_a = model_a.predict(X_test_estimated_a)



predictions_a.plot()


[[[ 1.95800000e+01 -1.00000000e+00 -9.47795909e-12  9.93587000e-01
   -1.13070214e-01]
  [ 0.00000000e+00 -9.65925826e-01  2.58819045e-01  9.93505698e-01
   -1.13782371e-01]
  [ 0.00000000e+00 -8.66025404e-01  5.00000000e-01  9.93423886e-01
   -1.14494469e-01]
  ...
  [ 2.02796000e+03 -7.07106781e-01 -7.07106781e-01  9.91772524e-01
   -1.28012739e-01]
  [ 1.09296000e+03 -8.66025404e-01 -5.00000000e-01  9.91680512e-01
   -1.28723591e-01]
  [ 3.94680000e+02 -9.65925826e-01 -2.58819045e-01  9.91587990e-01
   -1.29434377e-01]]

 [[ 0.00000000e+00  7.07106781e-01  7.07106781e-01 -9.68696752e-01
    2.48247060e-01]
  [ 0.00000000e+00  8.66025404e-01  5.00000000e-01 -9.68518564e-01
    2.48941341e-01]
  [ 0.00000000e+00  9.65925826e-01  2.58819045e-01 -9.68339878e-01
    2.49635494e-01]
  ...
  [ 0.00000000e+00 -3.28743159e-12  1.00000000e+00 -9.64850432e-01
    2.62799627e-01]
  [ 0.00000000e+00  2.58819045e-01  9.65925826e-01 -9.64661814e-01
    2.63491147e-01]
  [ 0.00000000e+00  5.0000000

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type Timestamp).

In [ ]:
#plotting the predictions
def plot_predictions1(model, X, y, start = 0, end =100):
    predictions = model.predict(X).flatten()
    df =  pd.DataFrame(data={"Predictions":predictions, "Actuals":y})
    plt.plot(df["Predictions"][start:end])
    plt.plot(df["Actuals"][start:end])
    return df, mae(y, predictions)